In [8]:
from openai import OpenAI
import json
import os
import chardet

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [9]:
def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."

def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [10]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [ ]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [11]:
#Prompting list
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Focus on important and likely scenarios, important scenario is the scenario that users are more likely to encounter it. 
- Minimize the appearance of rare scenarios. 
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""
FILTER_SYSTEM_PROMPT="""
You will be provide with a use case and a list of test scenario.
Based on information in the use case flow, define what test scenario is necessary to test the use case and remove duplicate test scenarios.
Remove test case test system load error that not be mentioned in use case such as: Load Failure, System Error,Network Error,Non-Existent ...(and use case not mentioned these flow)
Make sure test scenarios filted cover all the flow of use case and every scenarios in the response is unique.
Return scenario name only.
"""
SYSTEM_PROMPT_extract_flow ="""
Given use case specification flows.
Extract all the contents of flows into json format. 
If the use case's main scenario has a name other than 'Main flow', rename it to Main flow.
Other flows just keep the same name.
Skip all the information that not describe the flow action of use case(description, precondition,postcondition,...).
Return the flows in json format.
{
"Flow name":[ contents of corresponding flow],
}
For example: 
{
  "Main flow": [
    "Step 1: Learner fills in the username field by a valid username that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding username",
    "Step 3: Learner press \"Login\" button",
    "Step 4: System redirects learner to Home page"
  ],
  "Alternative flow 1: Login by email": [
    "At step 1 of the basic flow: Learner fills in the username field by a valid email that has been registered",
    "Step 2: Learner fills in the password field by the correct password for the corresponding email",
    "Go back to step 3 in the basic flow and continue with the steps from step 3"
  ]
}
"""

In [12]:
usecase_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Shopee(new)" #path to the directory store use case specification file
save_path = r"D:\Dissertation-GPT\evaluate\ResultSet\7.28\hotel" #path to the directory to save list of test scenario generated by method

In [13]:
usecase_name_list = [] # list of use case name in folder
project_name = os.path.basename(usecase_path)
for filename in os.listdir(usecase_path):
    usecase_name_list.append (filename.split(".txt")[0])

: 

In [14]:
#Test Design Phase
for usecase_name in usecase_name_list:
    # print(usecase_name)
    usecase_directlink = os.path.join(usecase_path,usecase_name+".txt")
    usecase_content = read_file_content(usecase_directlink) # store use case specification content

    #Use Case Analyzer
    promptExtractFlow = [
    { "role": "system", "content": SYSTEM_PROMPT_extract_flow},
    { "role": "user", "content": usecase_content}
    ]
    gpt_response = askJSON(promptExtractFlow, client, model)
    flows = json.loads(gpt_response)
    #get the response flow into parameters
    main_flow = flows['Main flow']
    alt_prompt=[]
    main_flow_prompt = "Main flow:"
    for i in main_flow:
        main_flow_prompt+="\n"+i
    for key,value in flows.items():
        if(key not in 'Main flow'):
            flow = key+ ":" 
            for i in value:
                flow += "\n"+ i
            alt_prompt.append(flow)

    #Condition Extractor
    promptExtractCondition = [
        { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
        { "role": "user", "content": usecase_content}
        ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    full_elements = json.loads(gpt_response)
    #filter to get condition of Text Field only
    condition_element = {key: value for key, value in full_elements.items() 
                                    if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
    print(condition_element)

    scenario_list = ""

    #Condition Scenario Generator
    if(len(condition_element)!= 0):
        promptScenarioForCondition = [
            { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
            { "role": "user", "content": usecase_content + '\nElement:' + str(condition_element)}
        ]
        condition_scenario_response = ask(promptScenarioForCondition, client, model)
        scenario_list = "\nCondition Scenario: " + condition_scenario_response
        print("\nCondition Scenario Generator: \n"+condition_scenario_response)

    #Main Flow Scenario Generator
    promptMainScenario = [
        { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt}
        ]
    main_gpt_response = ask(promptMainScenario, client, model)
    scenario_list += "\n Flow cover scenarios: " +  main_gpt_response
    print(main_flow_prompt)
    print("\nMain Flow Scenario Generator: \n"+main_gpt_response+"\n")

    #Sub Flow Scenario Generator
    for alt in alt_prompt:
        promptSubScenario = [
            { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
            { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
        ]
        sub_gpt_response = ask(promptSubScenario, client, model)
        print(alt)
        print("\nSub Flow Scenario Generator:"+sub_gpt_response)
        scenario_list+=  "\n" + sub_gpt_response
    
    scenario_refiner_input = "Use case: " + usecase_content +"\n Test scenarios: \n"
    scenario_refiner_input += scenario_list

    #Scenario Refiner
    promptFilter = [
        { "role": "system", "content": FILTER_SYSTEM_PROMPT},
        { "role": "user", "content": scenario_refiner_input}
        ]
    filter_gpt_response = ask(promptFilter, client, model)
    print("Scenario Refiner Response: \n" +filter_gpt_response)

    write_string_to_file(os.path.join(save_path,f"{usecase_name}.txt"), filter_gpt_response) #write scenario to save path directory

Use case name: Add item to cart
Description: This feature allows users to add items to their shopping cart for future purchase.
Actor: User
Preconditions: User is logged into their Shopee account.
Postconditions: 
The item is successfully added to the user's shopping cart.
The user can view and manage items in their shopping cart.
Basic Flow:
Step 1: User perform feature 'View a product detail'.
Step 2: User clicks the ""Add to Cart"" button.
Step 3: System confirms that the item has been added to the cart.
Step 5: System adds item to the cart and updates the cart icon to reflect the number of items in the cart.
Alterative flow: 
If product has many options, user choose one available option before adding product to cart.
User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 

Exception flow: 
- user cannot add product has many options without selecting one available option.
- user cannot add product with